**Description of scenario**: a transmitter tries to send data packets over a wireless channel with a bit error rate (BER) of 0.5. This means that each bit has a 50% chance of being corrupted during transmission. When an error occurs, the transmitter tries to resend the packet immediately.

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
import statistics

wait_times_1 = []
wait_times_2 = []

class network(object):
    def __init__(self, env, num_servers_1, service_rate_1):
        self.env = env
        self.server_1 = simpy.Resource(env, 1)
        self.service_rate_1 = service_rate_1

    def Queue_1(self, customer, error=False):
        if error:
            yield self.env.timeout(random.expovariate(self.service_rate_1) + random.expovariate(self.service_rate_1))
        else:
            yield self.env.timeout(random.expovariate(self.service_rate_1))


def go_to_queues(env, customer, network):
    arrival_time = env.now
    if random.random() >= 0.5:
        with network.server_1.request() as request:
            yield request
            yield env.process(network.Queue_1(customer, error=True))

        wait_times_2.append(env.now - arrival_time)
    else:
        with network.server_1.request() as request:
            yield request
            yield env.process(network.Queue_1(customer))

        wait_times_1.append(env.now - arrival_time)


def run_queues(env, arrival_rate, num_servers_1=1, service_rate_1=1):
    net = network(env, num_servers_1, service_rate_1)

    while True:
        yield env.timeout(random.expovariate(arrival_rate))

        customer =+ 1
        env.process(go_to_queues(env, customer, net))


def main(arrival_rate=1, service_rate_1=3, num_servers_1=1):
    # Run the simulation
    env = simpy.Environment()
    env.process(run_queues(env, arrival_rate, num_servers_1, service_rate_1))
    env.run(until=2*10**5)

    # View the results
    ave_1 = statistics.mean(wait_times_1)
    ave_2 = statistics.mean(wait_times_2)
    print(
        f"\nThe average wait time for customer without resending is {ave_1}.",
        f"\nThe average wait time for customer with resending is {ave_2}."
    )

In [ ]:
main()